In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#%cd drive/MyDrive/transformers_cl/ms_v2_root2_seed_3.zip (Unzipped Files)
# rename file name
# import os
# for seed in range(1, 6):
#   for pacing in ["root2", "geom_progression"]:
#     try:
#       os.rename(r'ms_v2_'+pacing+"_seed_"+str(seed), r'ms_v2_inverse_'+pacing+"_seed_"+str(seed))
#     except:
#       print("No", 'ms_v2_'+pacing+"_seed_"+str(seed))


In [16]:
# get test results
import glob
import os
import pandas as pd
from scipy import stats

# read the eval_results.txt file
def read_eval_results(result_path):
    #print(result_path)
    with open(result_path, 'r', encoding="utf8") as f:
        for line in f:
            line = line[:-1]
            map_val = float(line.split("map = ")[1])
            res = map_val
    return res

# all the test results are stored in these four folders
folders = ["ms_v2_base_cl", "ms_v2_dynamic", "ms_v2_noise", "mantis_noise"]
# convenient for the results to be transformed into pandas dataframe
keys = [set() for i in range(len(folders))]
# some are run with 5 seeds, some 3 seeds
nums_dic = {"ms_v2_base_cl":5, "ms_v2_dynamic":5, "ms_v2_noise":3, "mantis_noise":3}
res = {}
for i in range(len(folders)):
  # get the keys
  full = "drive/MyDrive/transformers_cl/"+ folders[i] +"/*/eval_results.txt"
  paths = glob.glob(full)
  for path in paths:
    key = path.split("/")[4]
    key_new = "_".join(key.split("_")[:-3])
    keys[i].add(key_new)

  res[folders[i]] = {}
  seeds = range(1, nums_dic[folders[i]]+1)
  for key_full in keys[i]:
    for seed in seeds:

      # for printing, to shorten the key
      if key_full[1] == "s":
        key = key_full[6:]
      else:
        key = key_full[7:]
      if folders[i] == 'ms_v2_dynamic':
        key = "_".join(key.split('_')[2:])

      if key not in res[folders[i]]:
        res[folders[i]][key] = []
      res[folders[i]][key].append(read_eval_results("drive/MyDrive/transformers_cl/"+ folders[i]+'/' + key_full + "_seed_" + str(seed) + '_test/eval_results.txt'))

In [17]:
import pandas as pd
res_dfs = {}
for k, v in res.items():
  res_dfs[k] = pd.DataFrame(v)
  num = res_dfs[k].shape[0]
  res_dfs[k].insert(0, "seed", list(range(1, num+1)))
  res_dfs[k] = res_dfs[k]

def add_avg_row(df_key):
  # add average row
  new = res_dfs[df_key].mean() 
  res_dfs[df_key] = res_dfs[df_key].append(new, ignore_index=True) 

def add_std_row(df_key):
  # add average row
  new = res_dfs[df_key].std() 
  res_dfs[df_key] = res_dfs[df_key].append(new, ignore_index=True) 

for k, v in res_dfs.items():
  add_avg_row(k)
  add_std_row(k)

In [15]:
# sort according to the average MAP
def sort_by_avg_row(df_key):
  res_dfs[df_key] = res_dfs[df_key].sort_values(by=res_dfs[df_key].shape[0]-2, axis=1, ascending=False)

# for convert df to latex
def rearrange_columns(df_key):
  # rearrange the columns
  new_row = {}
  s = list(res_dfs[df_key].columns)
  index = indexs[df_key]
  for i in range(len(s)):
    new_row[s[i]] = index[i] 
  res_dfs[df_key] = res_dfs[df_key].append(new_row, ignore_index=True)
  res_dfs[df_key] = res_dfs[df_key].sort_values(by=5, axis=1)
  # remove the assistant row
  res_dfs[df_key] = res_dfs[df_key].drop(5)


# for convert df to latex
def convert_df_to_latex(df_key):
  # round to 3 decimals
  res_dfs[df_key] = res_dfs[df_key].round(3)
  # # transpose df for printing out the results
  # res_dfs[df_key] = res_dfs[df_key].transpose()
  # convert to latex 
  #print(res_dfs[df_key].T.to_latex(index=False))


# this is an assistant row that we use to sort the columns
indexs = {"ms_v2_base_cl":[0,3,1,5,2,7,6,4]} # 'ms_v2_dynamic', 'ms_v2_noise', 'mantis_noise'
for k, v in res_dfs.items():
  if k != "ms_v2_base_cl":
    sort_by_avg_row(k)
    convert_df_to_latex(k)


In [5]:
res_dfs['ms_v2_base_cl']

,seed,scurve,root2,standard_training,inverse_root2,sigmoid,inverse_geom_progression,geom_progression
0,1.000000,0.707449,0.729284,0.718388,0.712907,0.718914,0.700880,0.728007
1,2.000000,0.680600,0.712546,0.707360,0.691873,0.704500,0.664110,0.714031
2,3.000000,0.704740,0.728022,0.726084,0.712673,0.721590,0.695787,0.727129
3,4.000000,0.708718,0.722472,0.702938,0.702712,0.721527,0.664796,0.723197
4,5.000000,0.691226,0.731774,0.727292,0.702857,0.724728,0.695849,0.730037
5,3.000000,0.698547,0.724820,0.716413,0.704604,0.718252,0.684285,0.724480
6,1.414214,0.010925,0.006851,0.009791,0.007782,0.007118,0.016299,0.005677


In [6]:
res_dfs['ms_v2_dynamic']

,0,1,2,3,4,5,6
seed,1.000,2.000,3.000,4.000,5.000,3.000,1.414
original_root_2,0.729,0.712,0.726,0.710,0.726,0.721,0.008
dynamic_38_root_2,0.729,0.705,0.726,0.716,0.727,0.720,0.009
dynamic_1_root_2,0.721,0.714,0.728,0.703,0.733,0.720,0.011
dynamic_34_root_2,0.728,0.703,0.736,0.704,0.723,0.719,0.013
dynamic_2_root_2,0.731,0.705,0.719,0.722,0.716,0.718,0.008
dynamic_3_root_2,0.732,0.707,0.724,0.702,0.726,0.718,0.012
dynamic_4_root_2,0.725,0.710,0.725,0.706,0.723,0.718,0.008
original_sd,0.722,0.708,0.718,0.707,0.730,0.717,0.009
dynamic_0_root_2,0.720,0.694,0.716,0.702,0.716,0.709,0.010


In [7]:
res_dfs["ms_v2_noise"]

,0,1,2,3,4
seed,1.000,2.000,3.000,2.000,0.816
root2,0.734,0.712,0.732,0.726,0.010
root2_l_995_r_05,0.726,0.714,0.734,0.725,0.008
root2_l_995_r_1,0.731,0.713,0.726,0.723,0.007
root2_l_99_r_1,0.729,0.711,0.729,0.723,0.008
root2_l_99_r_66,0.730,0.710,0.729,0.723,0.009
root2_l_995_r_66,0.731,0.711,0.726,0.723,0.009
root2_l_99_r_05,0.730,0.708,0.729,0.722,0.010
root2_l_999_r_1,0.724,0.704,0.734,0.721,0.012
root2_l_999_r_66,0.724,0.704,0.722,0.716,0.009


In [8]:
res_dfs["mantis_noise"]

,0,1,2,3,4
seed,1.000,2.000,3.000,2.000,0.816
root5_l_995_r_66,0.720,0.715,0.720,0.719,0.002
root5_l_99_r_1,0.720,0.712,0.720,0.717,0.004
root5_l_99_r_05,0.721,0.710,0.718,0.716,0.004
root5_l_995_r_05,0.719,0.712,0.719,0.716,0.003
standard_training,0.718,0.713,0.717,0.716,0.002
root5,0.720,0.708,0.720,0.716,0.006
root5_l_999_r_1,0.719,0.707,0.717,0.714,0.005
root5_l_99_r_66,0.720,0.708,0.714,0.714,0.005
root5_l_995_r_1,0.706,0.713,0.717,0.712,0.004


In [9]:
# Student's t-test on the ms_v2_base_cl
path_dataset = "drive/MyDrive/transformers_cl/"
folder = "ms_v2_base_cl"
res = {"seed": ['1','2','3','4','5']}
for seed in ['1','2','3','4','5']:
  baseline = pd.read_csv(path_dataset+folder+"/ms_v2_standard_training_seed_"+seed+"_test/aps_run_cl__seed_42", names=['ap'])
  for competing_pacing in res_dfs[folder].keys():
    p = ""
    if competing_pacing not in ["seed", "standard_training"]:
      competing = pd.read_csv(path_dataset+folder+'/ms_v2_'+competing_pacing+'_seed_'+seed+"_test/aps_run_cl__seed_42", names=['ap'])		
      statistic, pvalue = stats.ttest_rel(competing['ap'].values, baseline['ap'].values)
      if pvalue<=0.01 and statistic > 0 :
        p = ">, p<=0.01"
      elif pvalue<=0.05 and statistic > 0 :
        p = ">, p<=0.05"
      if pvalue<=0.01 and statistic < 0 :
        p = "<, p<=0.01"
      elif pvalue<=0.05 and statistic < 0 :
        p = "<, p<=0.05"

      p_key = competing_pacing + "_p_value"
      if p_key not in res:
        res[p_key] = []
      res[p_key].append(p)

pd.DataFrame(res)
    
# if competing_pacing not in res:
#   res[competing_pacing] = []
# res[competing_pacing].append(round(competing['ap'].mean(), 3))

,seed,scurve_p_value,root2_p_value,inverse_root2_p_value,sigmoid_p_value,inverse_geom_progression_p_value,geom_progression_p_value
0,1,"<, p<=0.01",">, p<=0.01",,,"<, p<=0.01",">, p<=0.05"
1,2,"<, p<=0.01",,"<, p<=0.01",,"<, p<=0.01",
2,3,"<, p<=0.01",,"<, p<=0.01",,"<, p<=0.01",
3,4,,">, p<=0.01",,">, p<=0.01","<, p<=0.01",">, p<=0.01"
4,5,"<, p<=0.01",,"<, p<=0.01",,"<, p<=0.01",


In [18]:
# Student's t-test on the ms_v2_dynamic folder
path_dataset = "drive/MyDrive/transformers_cl/"
folder = "ms_v2_dynamic"
res = {"seed": ['1','2','3','4','5']}
for seed in ['1','2','3','4','5']:
  baseline = pd.read_csv(path_dataset+folder+"/ms_v2_preds_dif_original_root_2_seed_"+seed+"_test/aps_run_cl__seed_42", names=['ap'])
  for competing_pacing in res_dfs[folder].keys():
    p = ""
    if competing_pacing not in ["seed", "original_root_2"]:
      competing = pd.read_csv(path_dataset+folder+'/ms_v2_preds_dif_'+competing_pacing+'_seed_'+seed+"_test/aps_run_cl__seed_42", names=['ap'])		
      statistic, pvalue = stats.ttest_rel(competing['ap'].values, baseline['ap'].values)
      if pvalue<=0.01 and statistic > 0 :
        p = ">, p<=0.01"
      elif pvalue<=0.05 and statistic > 0 :
        p = ">, p<=0.05"
      if pvalue<=0.01 and statistic < 0 :
        p = "<, p<=0.01"
      elif pvalue<=0.05 and statistic < 0 :
        p = "<, p<=0.05"

      p_key = competing_pacing + "_p_value"
      if p_key not in res:
        res[p_key] = []
      res[p_key].append(p)

pd.DataFrame(res)

,seed,dynamic_38_root_2_p_value,original_sd_p_value,dynamic_3_root_2_p_value,dynamic_2_root_2_p_value,dynamic_1_root_2_p_value,dynamic_4_root_2_p_value,dynamic_34_root_2_p_value,dynamic_0_root_2_p_value
0,1,,,,,"<, p<=0.05",,,"<, p<=0.01"
1,2,"<, p<=0.01",,,"<, p<=0.05",,,"<, p<=0.01","<, p<=0.01"
2,3,,"<, p<=0.05",,"<, p<=0.05",,,">, p<=0.01","<, p<=0.01"
3,4,,,"<, p<=0.05",">, p<=0.01",,,,"<, p<=0.05"
4,5,,,,"<, p<=0.01",,,,"<, p<=0.01"


In [19]:
# Student's t-test on the ms_v2_noise folder
path_dataset = "drive/MyDrive/transformers_cl/"
folder = "ms_v2_noise"
res = {"seed": ['1','2','3']}
for seed in ['1','2','3']:
  baseline = pd.read_csv(path_dataset+folder+"/ms_v2_root2_seed_"+seed+"_test/aps_run_cl__seed_42", names=['ap'])
  for competing_pacing in res_dfs[folder].keys():
    p = ""
    if competing_pacing not in ["seed", "root2"]:
      competing = pd.read_csv(path_dataset+folder+'/ms_v2_'+competing_pacing+'_seed_'+seed+"_test/aps_run_cl__seed_42", names=['ap'])		
      statistic, pvalue = stats.ttest_rel(competing['ap'].values, baseline['ap'].values)
      if pvalue<=0.01 and statistic > 0 :
        p = ">, p<=0.01"
      elif pvalue<=0.05 and statistic > 0 :
        p = ">, p<=0.05"
      if pvalue<=0.01 and statistic < 0 :
        p = "<, p<=0.01"
      elif pvalue<=0.05 and statistic < 0 :
        p = "<, p<=0.05"

      p_key = competing_pacing + "_p_value"
      if p_key not in res:
        res[p_key] = []
      res[p_key].append(p)

pd.DataFrame(res)

,seed,root2_l_99_r_1_p_value,root2_l_99_r_05_p_value,standard_training_p_value,root2_l_995_r_66_p_value,root2_l_999_r_66_p_value,root2_l_995_r_1_p_value,root2_l_99_r_66_p_value,root2_l_999_r_05_p_value,root2_l_999_r_1_p_value,root2_l_995_r_05_p_value
0,1,,,"<, p<=0.01",,"<, p<=0.01",,,"<, p<=0.01","<, p<=0.01","<, p<=0.05"
1,2,,,"<, p<=0.05",,"<, p<=0.05",,,"<, p<=0.01","<, p<=0.05",
2,3,,,"<, p<=0.01",,"<, p<=0.01",,,"<, p<=0.01",,


In [20]:
# Student's t-test on the mantis_noise folder
path_dataset = "drive/MyDrive/transformers_cl/"
folder = "mantis_noise"
res = {"seed": ['1','2','3']}
for seed in ['1','2','3']:
  baseline = pd.read_csv(path_dataset+folder+"/mantis_root5_seed_"+seed+"_test/aps_run_cl__seed_42", names=['ap'])
  for competing_pacing in res_dfs[folder].keys():
    p = ""
    if competing_pacing not in ["seed", "root5"]:
      competing = pd.read_csv(path_dataset+folder+'/mantis_'+competing_pacing+'_seed_'+seed+"_test/aps_run_cl__seed_42", names=['ap'])		
      statistic, pvalue = stats.ttest_rel(competing['ap'].values, baseline['ap'].values)
      if pvalue<=0.01 and statistic > 0 :
        p = ">, p<=0.01"
      elif pvalue<=0.05 and statistic > 0 :
        p = ">, p<=0.05"
      if pvalue<=0.01 and statistic < 0 :
        p = "<, p<=0.01"
      elif pvalue<=0.05 and statistic < 0 :
        p = "<, p<=0.05"

      p_key = competing_pacing + "_p_value"
      if p_key not in res:
        res[p_key] = []
      res[p_key].append(p)

pd.DataFrame(res)

,seed,root5_l_99_r_1_p_value,root5_l_995_r_66_p_value,standard_training_p_value,root5_l_999_r_66_p_value,root5_l_995_r_1_p_value,root5_l_999_r_1_p_value,root5_l_99_r_05_p_value,root5_l_99_r_66_p_value,root5_l_995_r_05_p_value
0,1,,,,"<, p<=0.01","<, p<=0.01",,,,
1,2,">, p<=0.01",">, p<=0.01",">, p<=0.01",,">, p<=0.01",,">, p<=0.05",,">, p<=0.01"
2,3,,,"<, p<=0.05","<, p<=0.01","<, p<=0.05","<, p<=0.05",,"<, p<=0.01",
